<a href="https://colab.research.google.com/github/sira-p/marmo-4b/blob/main/Marmo_4b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part One: Transfer Learning from Pre-trained Pose Detection Model

In [ ]:
#download training dataset
!rm -rf sample_data
!test -d train || (curl -o train.zip https://zenodo.org/records/5849371/files/marmoset-dlc-2021-05-07.zip?download=1 && unzip -q train.zip -d ./train && rm -f train.zip)

In [ ]:
#install ultralytics and deeplabcut2yolo packages via pip
!pip install ultralytics deeplabcut2yolo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 29.0 MB/s eta 0:00:00


In [ ]:
#obsolete
def mount_drive():
  import locale
  def getpreferredencoding(do_setlocale = True):
      return "UTF-8"
  locale.getpreferredencoding = getpreferredencoding

  from google.colab import drive
  drive.mount('/content/drive')

mount_drive()

In [ ]:
#load deeplabcut packages and specify input-output path
import deeplabcut2yolo as d2y

json_path = "./train/marmoset-dlc-2021-05-07/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/dlc_shuffle1_train.json"
csv_path = "./train/marmoset-dlc-2021-05-07/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/CollectedData_dlc.csv"
root_dir = "./train/marmoset-dlc-2021-05-07/labeled-data/"

#initiate conversion from deeplabcut(coco-like structure to yolov8 format)
d2y.convert(json_path, csv_path, root_dir, datapoint_classes=[0, 1], n_keypoint_per_datapoint=30, precision=6)

,height,file_name,width,id,normalized_coords,0_coords,data_0,0_bbox,0_x,0_y,0_w,0_h,1_coords,data_1,1_bbox,1_x,1_y,1_w,1_h
0,500,reachingvideo1_img01053.png,500,1690,"[0.44621944721940004, 0.4358964368964, 0.44990...","[0.44621944721940004, 0.4358964368964, 0.44990...",0.446219 0.435896 1 0.449900 0.490226 1 0.5097...,"[0.44621944721940004, 0.4245175839066, 0.70595...",0.576090,0.52095,0.259740,0.192864,"[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,None,0.000000,0.000000,0.000000,0.000000
1,300,reachingvideo1_img05399.png,400,8487,"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,None,0.000000,0.00000,0.000000,0.000000,"[0.4925, 0.32, 0.4425, 0.39666666666666667, 0....",0.492500 0.320000 1 0.442500 0.396667 1 0.5550...,"[0.4425, 0.32, 0.635, 0.7433333333333333]",0.538750,0.531667,0.192500,0.423333
2,500,reachingvideo1_img01711.png,500,2771,"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,None,0.000000,0.00000,0.000000,0.000000,"[None, None, 0.6443546453546, 0.25274625374619...",0 0 0 0.644355 0.252746 1 0.727605 0.292706 1 ...,"[0.5261395271396, 0.24275624375619997, 0.72760...",0.626872,0.600732,0.201465,0.715951
3,500,reachingvideo1_img01516.png,500,2403,"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,None,0.000000,0.00000,0.000000,0.000000,"[0.5178145188145999, 0.7322667332668, 0.556109...",0.517815 0.732267 1 0.556110 0.660672 1 0.4495...,"[0.3787293701172, 0.3243413253414, 0.562769563...",0.470749,0.528304,0.184040,0.407925
4,600,reachingvideo1_img02382.png,600,4057,"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,None,0.000000,0.00000,0.000000,0.000000,"[0.49666666666666665, 0.525, 0.543333333333333...",0.496667 0.525000 1 0.543333 0.498333 1 0.4616...,"[0.4166666666666667, 0.27166666666666667, 0.55...",0.487500,0.398333,0.141667,0.253333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3959,400,reachingvideo1_img06891.png,400,10630,"[0.51, 0.6, 0.545, 0.5425, 0.465, 0.5625, 0.50...","[0.51, 0.6, 0.545, 0.5425, 0.465, 0.5625, 0.50...",0.510000 0.600000 1 0.545000 0.542500 1 0.4650...,"[0.4425, 0.3925, 0.545, 0.6]",0.493750,0.49625,0.102500,0.207500,"[0.5775, 0.505, 0.5925, 0.56, 0.62, 0.4875, 0....",0.577500 0.505000 1 0.592500 0.560000 1 0.6200...,"[0.5775, 0.4375, 0.6925, 0.56]",0.635000,0.498750,0.115000,0.122500
3960,400,reachingvideo1_img06506.png,400,10245,"[0.73, 0.645, 0.805, 0.585, 0.6725, 0.5875, 0....","[0.73, 0.645, 0.805, 0.585, 0.6725, 0.5875, 0....",0.730000 0.645000 1 0.805000 0.585000 1 0.6725...,"[0.4425, 0.4, 0.805, 0.7125]",0.623750,0.55625,0.362500,0.312500,"[0.295, 0.2875, 0.255, 0.35, 0.3625, 0.3125, 0...",0.295000 0.287500 1 0.255000 0.350000 1 0.3625...,"[0.255, 0.2875, 0.535, 0.6025]",0.395000,0.445000,0.280000,0.315000
3961,500,reachingvideo1_img04631.png,500,7184,"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,None,0.000000,0.00000,0.000000,0.000000,"[0.3606403596404, 0.6573416583416, 0.423080609...",0.360640 0.657342 1 0.423081 0.693122 1 0.3871...,"[0.32900532800540006, 0.4059264069264, 0.68826...",0.508633,0.549524,0.359256,0.287196
3962,500,reachingvideo1_img01332.png,500,2121,"[0.5278045288045999, 0.6590066600066, 0.561104...","[0.5278045288045999, 0.6590066600066, 0.561104...",0.527805 0.659007 1 0.561105 0.602397 1 0.4711...,"[0.44592902565000003, 0.42323251140120005, 0.5...",0.518502,0.54112,0.145146,0.235774,"[None, None, None, None, None, None, None, Non...",0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
#define function for create configuration file (.yaml)
def create_data_yml(train_path, val_path, kpt_shape, flip_idx, nc, names, output_path):
  data = {
    "train": train_path,
    "val": val_path,
    "kpt_shape": kpt_shape,
    "flip_idx": flip_idx,
    "nc": nc,
    "names": names
  }

  with open(output_path, "w") as f:
    import yaml
    yaml.dump(data, f, default_flow_style=None, sort_keys=False)

In [ ]:
#defining variables for file path and configuration for configuration file (.yaml)
train_path = "/content/train/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1"
val_path = "/content/train/marmoset-dlc-2021-05-07/labeled-data/refinement1"
data_path = "/content/data.yml"
shape = [15, 3]
flip_index = [0, 3, 2, 1, 6, 7, 4, 5, 9, 8, 11, 10, 12, 13, 14]
nc = 2
names = ["B", "W"]

#loading function create_data_yml to create configuration file (.yaml)
create_data_yml(train_path, val_path, shape, flip_index, nc, names, data_path)

In [ ]:
#loading YOLOv8 packages
from ultralytics import YOLO

#load YOLOv8 pose estimation pretrained model
model = YOLO("yolov8n-pose.pt").load('yolov8n-pose.pt')
#fine tune the pretrained model with the marmoset dataset from deeplabcut
model.train(data="data.yml", epochs=50, imgsz=640)

100%|██████████| 6.52M/6.52M [00:00<00:00, 132MB/s]


Transferred 397/397 items from pretrained weights
Ultralytics YOLOv8.2.90 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=data.yml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=yolov8n-pose.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=

100%|██████████| 755k/755k [00:00<00:00, 19.9MB/s]


Overriding model.yaml kpt_shape=[17, 3] with kpt_shape=[15, 3]
Overriding model.yaml nc=1 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True

100%|██████████| 6.25M/6.25M [00:00<00:00, 113MB/s]


...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/train/marmoset-dlc-2021-05-07/labeled-data/refinement1... 326 images, 98 backgrounds, 0 corrupt: 100%|██████████| 424/424 [00:01<00:00, 386.28it/s]

val: New cache created: /content/train/marmoset-dlc-2021-05-07/labeled-data/refinement1.cache


Plotting labels to runs/pose/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0005), 72 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/pose/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


       1/50      2.57G      1.605      10.52     0.6476      2.522      1.553         24        640:  97%|█████████▋| 297/306 [02:09<00:03,  2.29it/s]


KeyboardInterrupt: 

In [ ]:
#move files to a new directory
!mv /content/runs/pose/train /content/drive/MyDrive/marmo-4b/

Optionally, train a one-class pose detection model.

In [ ]:
#load deeplabcut packages and specify input-output path
import deeplabcut2yolo as d2y

json_path = "./train/marmoset-dlc-2021-05-07/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/dlc_shuffle1_train.json"
csv_path = "./train/marmoset-dlc-2021-05-07/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/CollectedData_dlc.csv"
root_dir = "./train/marmoset-dlc-2021-05-07/labeled-data/"

#initiate conversion from deeplabcut(coco-like structure to yolov8 format)
d2y.convert(json_path, csv_path, root_dir, datapoint_classes=[0, 0], n_keypoint_per_datapoint=30, precision=6)

In [ ]:
#defining variables for file path and configuration for configuration file (.yaml)
train_path = "/content/train/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1"
val_path = "/content/train/marmoset-dlc-2021-05-07/labeled-data/refinement1"
data_path = "/content/data.yml"
shape = [15, 3]
flip_index = [0, 3, 2, 1, 6, 7, 4, 5, 9, 8, 11, 10, 12, 13, 14]

#initiate conversion from deeplabcut(coco-like structure to yolov8 format) by specifying class as 1, the model is trained only with one marmoset
create_data_yml(train_path, val_path, shape, flip_index, 1, ["marmoset"], data_path)

In [ ]:
#loading YOLOv8 packages
from ultralytics import YOLO

#load YOLOv8 pose estimation pretrained model
model_one_class = YOLO("yolov8n-pose.pt").load('yolov8n-pose.pt')
#fine tune the pretrained model with the marmoset dataset from deeplabcut
model_one_class.train(data="data.yml", epochs=50, imgsz=640)

Transferred 397/397 items from pretrained weights
Ultralytics YOLOv8.2.87 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=data.yml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=yolov8n-pose.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=

train: Scanning /content/train/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1.cache... 3638 images, 1254 backgrounds, 0 corrupt: 100%|██████████| 4892/4892 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/train/marmoset-dlc-2021-05-07/labeled-data/refinement1... 326 images, 98 backgrounds, 0 corrupt: 100%|██████████| 424/424 [00:01<00:00, 394.64it/s]


val: New cache created: /content/train/marmoset-dlc-2021-05-07/labeled-data/refinement1.cache
Plotting labels to runs/pose/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0005), 72 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/pose/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


       1/50      2.63G       1.78       10.9     0.6858       2.05      1.696         28        640:  48%|████▊     | 148/306 [01:05<01:13,  2.14it/s]

In [ ]:
#move trained files to a new directory
!mv /content/runs/pose/train /content/drive/MyDrive/marmo-4b-one/

# Part Two: SAM Mask Generation Using Points from Pose Detection

In [ ]:
#workaround for notebook git clone bug
import locale
locale.getpreferredencoding = lambda: "UTF-8"
#download pretrained pose detection model (from part 1) from github
!git clone https://github.com/sira-p/marmo-4b/
!rm -rf sample_data
#download segmentanything from github
!find sam_vit_h.pth || curl -o sam_vit_h.pth https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
#download the test dataset
!test -d test || (curl -o test.zip https://zenodo.org/records/8437121/files/marmoset-dlc-2021-05-07.zip?download=1 && unzip -q test.zip -d ./test && rm -f test.zip)
#install segmentanything and ultralytics from pip
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install ultralytics

Cloning into 'marmo-4b'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 28 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 5.89 MiB | 7.94 MiB/s, done.
Resolving deltas: 100% (7/7), done.
find: ‘sam_vit_h.pth’: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2445M  100 2445M    0     0   120M      0  0:00:20  0:00:20 --:--:--  200M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  776M  100  776M    0     0  20.2M      0  0:00:38  0:00:38 --:--:-- 21.4M
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-gkvauid5
  Running command git clone --filter=blob:none --

In [ ]:
#loading dependecies
from ultralytics import YOLO
import glob
from segment_anything import SamPredictor, sam_model_registry
import numpy as np
import torch
import cv2
import os
from tqdm import tqdm

In [ ]:
#define the Marmo-4b model class
class Marmo4b:
  #defining the color for the masks and skeletons connections
  COLORS = [
      (255, 0, 0),
      (0, 255, 0),
      (0, 0, 255),
      (255, 255, 0),
      (255, 0, 255),
      (0, 255, 255),
    ]
  SKELETON = [
    (0, 1),
    (0, 2),
    (0, 3),
    (3, 7),
    (7, 8),
    (5, 12),
    (13, 14),
    (11, 14),
    (3, 9),
    (9, 10),
    (3, 4),
    (4, 5),
    (6, 13),
    (6, 11),
  ]
  CONFIDENCE_THRESHOLD = 0.5

#defining initiation function for SAM
  def __init__(self, pose_model_path, sam_checkpoint_path):
    self.pose_model = YOLO(pose_model_path)
    sam = sam_model_registry["default"](checkpoint=sam_checkpoint_path)
    sam = sam.to(device='cuda' if torch.cuda.is_available() else 'cpu')
    self.mask_predictor = SamPredictor(sam)

#defining function for color masks
  @staticmethod
  def apply_color_masks(image, masks):
    for i, mask in enumerate(masks):
      mask = cv2.cvtColor((mask[0].squeeze() * 255).astype('uint8'), cv2.COLOR_GRAY2BGR)
      color_mask = np.zeros_like(image)
      color_mask[:,:] = Marmo4b.COLORS[i]
      color_mask[mask == 0] = 0

      image = cv2.addWeighted(image, 0.7, color_mask, 0.3, 0)
    return image

#defining function for joint points
  @staticmethod
  def draw_joint(image, points):
    for i in range(len(points)):
      for x, y in points[i]:
        image = cv2.circle(image, (int(x), int(y)), 2, Marmo4b.COLORS[i], -1)
    return image

#defining function to draw skeleton connections
  @staticmethod
  def draw_skeleton(image, points, masks):
    overlay = image.copy()
    for i in range(len(points)):
      mask = masks[i]
      point = points[i]
      for start_i, end_i in Marmo4b.SKELETON:
        if mask[start_i] and mask[end_i]:
          # If confident, then draw
          overlay = cv2.line(overlay, tuple(point[start_i].astype(int)), tuple(point[end_i].astype(int)), Marmo4b.COLORS[i], 2)

    image = cv2.addWeighted(image, 0.3, overlay, 0.7, 0)

    return image

#defining funtion for drawing bounding boxes
  @staticmethod
  def is_within_box(bboxes, points):
    mask = np.zeros((points.shape[0], points.shape[1]), dtype=bool)
    for bbox in bboxes:
      in_x = (points[:, :, 0] >= bbox[0]) & (points[:, :, 0] <= bbox[2])
      in_y = (points[:, :, 1] >= bbox[1]) & (points[:, :, 1] <= bbox[3])

      mask = mask | (in_x & in_y)
    return mask

#defining function for using both models together
  def predict_image(self, image, mode="points"):
    """
    Args:
      image (OpenCV image object)
    """
    if image is None:
      raise ValueError("Image not provided")

    MODES = ["points", "box"]
    if not mode in MODES:
      raise ValueError(f"Invalid mode of operation: use {' '.join(MODES)}")

    pose_result = self.pose_model(image)[0]

    joint_points = pose_result.keypoints.xy.cpu().numpy()
    boxes = pose_result.boxes.xyxy.cpu().numpy()
    point_confidences = pose_result.keypoints.conf.cpu().numpy()
    # box_mask = is_within_box(boxes, joint_points)
    confidence_mask = point_confidences > self.CONFIDENCE_THRESHOLD
    zero_mask = ~np.all(joint_points[:, :, :] == np.array([0, 0]), axis=2)
    point_mask = zero_mask & confidence_mask
    input_points = np.where(np.expand_dims(point_mask, axis=2), joint_points, 0)

    self.mask_predictor.set_image(image)

    if mode == "points":
      input_labels = point_mask.astype(int)
      sam_masks = [self.mask_predictor.predict(point_coords=point, point_labels=label, multimask_output=False) for point, label in zip(input_points, input_labels)]

    if mode == "box":
      sam_masks = [self.mask_predictor.predict(box=box, multimask_output=False) for box in boxes]

    image = self.apply_color_masks(image, sam_masks)
    image = self.draw_joint(image, input_points)
    image = self.draw_skeleton(image, joint_points, point_mask)

    return image

  def predict_images(self, image_paths, save_path, mode="points"):
    os.makedirs(save_path, exist_ok=True)

    for image_path in tqdm(image_paths):
      image = cv2.imread(image_path)
      result = self.predict_image(image, mode)
      cv2.imwrite(save_path + image_path.split("/")[-1], result)

In [ ]:
#initializing the model
model = Marmo4b("marmo-4b/marmo-4b.pt", "/content/sam_vit_h.pth")

In [ ]:
#single image demo
image = cv2.imread("/content/test/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/img01310.png")
result = model.predict_image(image, mode="box")
cv2.imwrite("processed.png", result)


0: 640x640 1 W, 14.0ms
Speed: 3.4ms preprocess, 14.0ms inference, 732.7ms postprocess per image at shape (1, 3, 640, 640)


True

In [ ]:
#setting image path for prediction
image_paths = glob.glob("/content/test/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/*.png")
#batch prediction
model.predict_images(image_paths, "/content/predict/", mode="box")

  0%|          | 0/2108 [00:00<?, ?it/s]


0: 640x640 1 B, 9.1ms
Speed: 7.4ms preprocess, 9.1ms inference, 1303.8ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 1/2108 [00:05<3:17:12,  5.62s/it]


0: 640x640 1 B, 14.0ms
Speed: 6.0ms preprocess, 14.0ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 2/2108 [00:07<2:00:59,  3.45s/it]

...


0: 480x640 1 W, 8.2ms
Speed: 2.7ms preprocess, 8.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)


  2%|▏         | 39/2108 [01:18<1:09:11,  2.01s/it]


0: 640x640 1 W, 10.3ms
Speed: 3.5ms preprocess, 10.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


  2%|▏         | 40/2108 [01:20<1:09:23,  2.01s/it]


KeyboardInterrupt: 

In [ ]:
#image to video conversion function
def images_to_video(image_directory, video_name, fps=30, target_size=(500, 500)):

  image_paths = glob.glob(image_directory + "*.png")
  image_paths.sort()

  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  video = cv2.VideoWriter(video_name, fourcc, fps, target_size)

  for image_path in image_paths:
    frame = cv2.imread(image_path)
    resized_frame = cv2.resize(frame, target_size)
    video.write(resized_frame)

  video.release()

In [ ]:
#convert image to video
images_to_video("/content/predict/", "myvideo.mp4")

# Part Three: Implementing Interface

In [ ]:
#installing Gradio via pip
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existin

In [ ]:
#loading prerequisites
import gradio as gr
from PIL import Image
import os

In [ ]:
#defining function for batch prediction (multiple images)
def batch_predict(image_paths, mode, model):
  temp_dir = "temp_results/"
  os.makedirs(temp_dir, exist_ok=True)

  model.predict_images(image_paths, temp_dir, mode)
  result_paths = glob.glob(temp_dir + "*")

  return [Image.open(result_path) for result_path in result_paths]

#defining gradio's interface
def launch_interface(model):
  gr.Interface(
      fn=lambda image_list, mode: batch_predict(image_list, mode, model),
      inputs=[
          gr.File(label="Upload images", file_count="multiple", type="filepath"),
          gr.Radio(choices=["points", "box"], label="Segmentation mode"),
      ],
      outputs=gr.Gallery(label="Processed Images"),
      title="Pose Estimation and Instance Segmentation",
      description="Upload images and select the mode (points or box) to predict poses and masks."
  ).launch(debug=True)

In [ ]:
#launch gradio webapp
launch_interface(model)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://25825e035c4385e36b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


  0%|          | 0/1 [00:00<?, ?it/s]


0: 640x640 1 W, 45.6ms
Speed: 5.0ms preprocess, 45.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 0/1 [00:00<?, ?it/s]


0: 640x640 1 W, 10.2ms
Speed: 3.9ms preprocess, 10.2ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████| 1/1 [00:01<00:00,  1.94s/it]
